Model download link: https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5/

In [3]:
from imageai.Detection import ObjectDetection
import os

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("models/resnet50_coco_best_v2.1.0.h5")
detector.loadModel()

In [4]:
import pickle

objects_detected_w_proba = dict()
dir_name = "images"
images_left = len(os.listdir(dir_name))

for filename in os.listdir(dir_name)[:5]:
    if filename == ".gitignore":
        continue

    detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename, output_type="array")[1]
    # detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename,
    #                                              output_image_path=f"{dir_name}/new_" + filename)

    objects_detected_w_proba[filename] = []

    for eachObject in detections:
        objects_detected_w_proba[filename].append((eachObject["name"], eachObject["percentage_probability"]))
        # print(filename, (eachObject["name"], eachObject["percentage_probability"]))

    images_left -= 1
    print("images left:", images_left)

    if images_left % 100 == 0:
        with open("data/thumbnails_objects_dict.pkl", "wb") as f:
            pickle.dump(objects_detected_w_proba, f)

ValueError: Ensure you specified correct input image, input type, output type and/or output image path 

In [ ]:
with open("data/thumbnails_objects_dict.pkl", "wb") as f:
    pickle.dump(objects_detected_w_proba, f)

In [1]:
import pickle

# detecions_dict = dict()
with open("data/thumbnails_objects_dict.pkl", "rb") as f:
    objects_detected_w_proba = dict(pickle.load(f))
print(objects_detected_w_proba)

{'G9MJjAArKBk.jpg': [('person', 73.3136236667633)], 'dEHZNLzXULA.jpg': [('person', 90.70668816566467)], 'E9MxTo1W2PY.jpg': [('bird', 54.18645143508911)], 'h4qG1FpMzCY.jpg': [], 'HK_1PIONSlw.jpg': [('person', 95.82610726356506), ('person', 89.2713189125061)], 'O3kuDT1tzC4.jpg': [('person', 88.89310956001282), ('person', 61.497193574905396)], 'orWtpK0sKTs.jpg': [('person', 93.86693239212036), ('person', 84.57270860671997)], 'Fb3NjxoqIJg.jpg': [('person', 89.86856937408447), ('person', 78.04118394851685)], 'i3XCb6OaCt8.jpg': [('person', 95.93670964241028), ('person', 52.00619101524353)], 'DBkXemt401I.jpg': [('person', 98.33009243011475)], 'cy89Q4hp-JE.jpg': [('person', 52.56825089454651)], 'ioxhcuNSxZk.jpg': [], 'nAUwKeO93bY.jpg': [('person', 93.12952756881714), ('person', 84.69846248626709), ('person', 78.54865789413452), ('person', 72.36691117286682), ('person', 71.97596430778503), ('person', 67.97313690185547), ('person', 63.58312368392944), ('person', 56.329506635665894)], 'KZPBTTnqnA

In [2]:
objects_detected = dict()
counts = dict()

for key in objects_detected_w_proba.keys():
    objects_detected[key] = dict()

    for name, _ in objects_detected_w_proba[key]:
        objects_detected[key][name] = objects_detected[key].get(name, 0) + 1
        counts[name] = counts.get(name, 0) + 1

counts = dict(reversed(sorted(counts.items(), key=lambda item: item[1])))
top_popular_objects = list(counts.keys())[:10]

# print("objects counts", objects_detected)
# print("counts", counts)
print("10 most common objects:", top_popular_objects)

10 most common objects: ['person', 'car', 'cell phone', 'tie', 'cup', 'bottle', 'chair', 'truck', 'tv', 'dog']


In [4]:
import pandas as pd

df = pd.DataFrame.from_dict(objects_detected).transpose()
df = df.fillna(0)
df = df.astype('int32')

df.to_csv("data/thumbnails_objects_vectors.csv")